In [2]:
# download uniprot ID mapping
! wget --directory-prefix download ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping.dat.gz

--2015-04-14 09:50:50--  ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping.dat.gz
           => 'download/idmapping.dat.gz'
Resolving ftp.uniprot.org... 141.161.180.197
Connecting to ftp.uniprot.org|141.161.180.197|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/uniprot/current_release/knowledgebase/idmapping ... done.
==> SIZE idmapping.dat.gz ... 3789511248
==> PASV ... done.    ==> RETR idmapping.dat.gz ... done.
Length: 3789511248 (3.5G) (unauthoritative)

idmapping.dat.gz     28%[=====>                ]   1014M  --.-KB/s   in 46m 2s 

2015-04-14 10:36:53 (376 KB/s) - Data connection: Operation timed out; ^C


### idmapping.dat [documentation](ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/README)

1) idmapping.dat
This file has three columns, delimited by tab:
1. UniProtKB-AC 
2. ID_type 
3. ID

where ID_type is the database name as appearing in UniProtKB cross-references, 
and as supported by the ID mapping tool on the UniProt web site, 
http://www.uniprot.org/mapping and where ID is the identifier in 
that cross-referenced database.

In [4]:
import os
import csv
import io

In [8]:
def generate_idmapping(path):
    """Returns a generator of idmapping.dat.gz rows."""
    read_file = open(path, 'rb')
    text = io.TextIOWrapper(read_file)
    reader = csv.reader(text, delimiter='\t')
    for row in reader:
        yield row
    read_file.close()

In [9]:
path = os.path.join('download', 'idmapping.dat.gz')
mapping_generator = generate_idmapping(path)

In [10]:
extract = {'GeneID', 'HGNC'}
mappings = {target: set() for target in extract}

for accession, target, target_id in mapping_generator:
    if target not in extract:
        continue
    mappings[target].add((accession, target_id))

for target, mapset in mappings.items():
    path = os.path.join('data', 'map', '{}.tsv'.format(target))
    write_file = open(path, 'w')
    writer = csv.writer(write_file, delimiter='\t')
    writer.writerow(['uniprot', target])
    writer.writerows(sorted(mapset))
    write_file.close()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte